In [1]:
import arviz as az
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pymc as pm
import pytensor.tensor as pt
import requests
import statsmodels.api as sm
import warnings
import xarray as xr
from pathlib import Path
import urllib.request

In [2]:
countries_iso = ['BEL', 'CHE', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'ITA', 'NLD', 'NOR', 'PRT', 'SWE']
countries_names = ['Belgium', 'Switzerland', 'Denmark', 'Spain', 'Finland', 'France', 'United Kingdom', 'Italy', 'Netherlands', 'Norway', 'Portugal', 'Sweden']

name_to_iso = dict(zip(countries_names, countries_iso))

Jordà-Schularick-Taylor Macrohistory Database:
- https://www.macrohistory.net/app/download/9834512569/JSTdatasetR6.xlsx?t=1763503850

Long-Term Productivity Database:
- https://www.longtermproductivity.com/download.html

SIPRI Military Expenditure Database:
- https://www.sipri.org/databases/milex

In [3]:
data_path0 = Path.cwd().parent / "data raw" / "JSTdatasetR6.xlsx"
print("Macro dataset")
print("Full file path: ",data_path0)
print("File exists: ",data_path0.exists())
print()

data_path1 = Path.cwd().parent / "data raw" / "BCLDatabase_online_v2.6.xlsx"
print("TFP dataset")
print("Full file path: ",data_path1)
print("File exists: ",data_path1.exists())
print()

data_path2 = Path.cwd().parent / "data raw" / "SIPRI-Milex-data-1949-2024_2.xlsx"
print("Defense spending dataset")
print("Full file path: ",data_path2)
print("File exists: ",data_path2.exists())

Macro dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/JSTdatasetR6.xlsx
File exists:  True

TFP dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/BCLDatabase_online_v2.6.xlsx
File exists:  True

Defense spending dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/SIPRI-Milex-data-1949-2024_2.xlsx
File exists:  True


In [4]:
df_macro = pd.read_excel(data_path0)

In [5]:
#Real in constant 1990 USD (use 1990 exchange rate)
df_macro["gdp_real_lcu_1990"] = df_macro["gdp"] * 100 / df_macro["cpi"]
df_macro["revenue_real_lcu_1990"] = df_macro["revenue"] * 100 / df_macro["cpi"]
df_macro["expenditure_real_lcu_1990"] = df_macro["expenditure"] * 100 / df_macro["cpi"]

xrusd_1990 = (
    df_macro.loc[df_macro["year"].eq(1990)]
      .groupby("iso", as_index=False)["xrusd"]
      .mean()
      .rename(columns={"xrusd": "xrusd_1990"})
)

df_macro = df_macro.merge(xrusd_1990, on="iso", how="left")

df_macro["gdp_real_usd_1990"] = df_macro["gdp_real_lcu_1990"] / df_macro["xrusd_1990"]
df_macro["revenue_real_usd_1990"] = df_macro["revenue_real_lcu_1990"] / df_macro["xrusd_1990"]
df_macro["expenditure_real_usd_1990"] = df_macro["expenditure_real_lcu_1990"] / df_macro["xrusd_1990"]

In [6]:
df_macro["rgdp_pc"] = df_macro["gdp_real_usd_1990"]/df_macro["pop"] #Real GDP per capita
df_macro["def_gdp"] = 100*(df_macro["revenue_real_usd_1990"] -
                      df_macro["expenditure_real_usd_1990"])/df_macro["gdp_real_usd_1990"] #deficit as percent of gdp
df_macro["gov_pc"] = df_macro["expenditure_real_usd_1990"]/df_macro["pop"] #real government spending per capita
df_macro["bill_rate"] = df_macro["bill_rate"]*100 #convert bill rate to %
df_macro["debtgdp"] = df_macro["debtgdp"]*100 #convert debt to %
df_macro["nx_gdp"] = 100*(df_macro["exports"]-df_macro["imports"])/df_macro["gdp"]

In [7]:
df_macro = df_macro[["iso", #country code
                      "year", #year
                      "rgdp_pc", #Real GDP per capita
                      "gov_pc", #Real government spending per capita
                      "bill_rate", #Fed funds rate
                      "def_gdp", #deficit as percent of gdp
                      "debtgdp", #debt as percent of gdp
                      "nx_gdp" #trade balance as percent of GDP
                     ]]

In [8]:
df_tfp = pd.read_excel(data_path1, sheet_name=3)

df_tfp = df_tfp.rename(columns={df_tfp.columns[0]: "year"}).copy()
df_tfp = df_tfp.iloc[:, :-2]
df_tfp = df_tfp.melt(id_vars="year", var_name="iso", value_name="tfp")

In [9]:
df_mil = pd.read_excel(data_path2, sheet_name=6, skiprows=5, na_values=["..."])
df_mil = df_mil.rename(columns={df_mil.columns[0]: "iso"}).drop(columns=df_mil.columns[1])

df_mil = df_mil.melt(id_vars="iso", var_name="year", value_name="mil_gdp")
df_mil["iso"] = df_mil["iso"].replace(name_to_iso)
df_mil["mil_gdp"] = pd.to_numeric(df_mil["mil_gdp"], errors="coerce")
df_mil["mil_gdp"] = df_mil["mil_gdp"]*100

In [10]:
df = df_macro.merge(
    df_tfp,
    on=["year", "iso"],
    how="left"
)

df = df.merge(
    df_mil,
    on=["year", "iso"],
    how="left"
)

df["iso"] = df["iso"].astype("string")

df = df.loc[df["iso"].isin(countries_iso)].copy()

In [11]:
# sanity: drop rows missing iso/year
df = df.dropna(subset=["iso", "year"])

# ensure unique iso-year (if duplicates exist, decide how to aggregate)
dups = df.duplicated(["iso", "year"])
if dups.any():
    df = (df.groupby(["iso", "year"], as_index=False)
            .mean(numeric_only=True))  # or pick .first(), etc.

In [12]:
out = df.loc[(df["year"] >= 1950) & df.isna().any(axis=1)]
na_cols = out.columns[out.isna().any()].tolist()
out[["year","iso"] + na_cols]

,year,iso,bill_rate,def_gdp,debtgdp,mil_gdp
231,1950,BEL,1.400000,-5.179133,73.688900,NaN
232,1951,BEL,1.400000,-3.272222,64.479400,NaN
233,1952,BEL,1.400000,-5.771846,66.252200,NaN
261,1980,BEL,14.080000,-9.097519,NaN,3.250195
262,1981,BEL,15.250000,-14.285372,NaN,3.366049
533,1950,CHE,2.313333,1.451770,66.925450,NaN
534,1951,CHE,2.310000,-0.120381,64.483881,NaN
535,1952,CHE,2.311667,-0.822276,62.900016,NaN
536,1953,CHE,2.311667,0.333506,60.655782,NaN
537,1954,CHE,2.310000,1.227163,57.509509,NaN


In [13]:
df = df.loc[df.year >= 1950]
df = df.sort_values(["iso", "year"])

cols_to_interp = ["bill_rate", "mil_gdp", "def_gdp", "debtgdp"]  # your cols

df[cols_to_interp] = (
    df.groupby("iso", group_keys=False)[cols_to_interp]
      .apply(lambda g: g.interpolate(limit_area="inside"))
)

In [14]:
out = df.loc[(df["year"] >= 1950) & df.isna().any(axis=1)]
na_cols = out.columns[out.isna().any()].tolist()
out[["year","iso"] + na_cols]

,year,iso,def_gdp,debtgdp,mil_gdp
231,1950,BEL,-5.179133,73.688900,NaN
232,1951,BEL,-3.272222,64.479400,NaN
233,1952,BEL,-5.771846,66.252200,NaN
533,1950,CHE,1.451770,66.925450,NaN
534,1951,CHE,-0.120381,64.483881,NaN
535,1952,CHE,-0.822276,62.900016,NaN
536,1953,CHE,0.333506,60.655782,NaN
537,1954,CHE,1.227163,57.509509,NaN
538,1955,CHE,0.937325,53.778543,NaN
539,1956,CHE,1.925250,52.368122,NaN


In [15]:
df = df.dropna()

In [16]:
out = df.loc[(df["year"] >= 1950) & df.isna().any(axis=1)]
na_cols = out.columns[out.isna().any()].tolist()
out[["year","iso"] + na_cols]

,year,iso


In [17]:
vars_keep = ["rgdp_pc","gov_pc","bill_rate","def_gdp","debtgdp","tfp","mil_gdp","nx_gdp"]

# logs (only where strictly positive)
df["log_rgdp_pc"] = np.log(df["rgdp_pc"])
df["log_gov_pc"]  = np.log(df["gov_pc"])
df["log_tfp"]     = np.log(df["tfp"])

# ratios/rates typically already in percent points (just keep)
df["tbill"]    = df["bill_rate"]
df["def_gdp"]  = df["def_gdp"]
df["debt_gdp"] = df["debtgdp"]
df["mil_gdp"]  = df["mil_gdp"]
df["nx_gdp"]   = df["nx_gdp"]

model_vars = ["log_rgdp_pc","log_gov_pc","log_tfp","tbill","def_gdp","debt_gdp","mil_gdp","nx_gdp"]

In [18]:
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.sort_values(["iso", "year"])
df = df.set_index(["iso", "year"])

In [19]:
g = df["mil_gdp"].groupby(level=0)
df["mil_delta"] = g.shift(-1) - df["mil_gdp"]

In [20]:
df = df[["mil_delta","log_rgdp_pc","log_gov_pc","log_tfp","tbill","def_gdp","debt_gdp"]]
df = df.dropna()

In [21]:
df

mil_delta  log_rgdp_pc  log_gov_pc   log_tfp     tbill   def_gdp  \
iso year                                                                     
BEL 1953   0.017696     1.949283    0.487863  1.440028  1.400000 -4.360491   
    1954  -1.096842     1.971972    0.490545  1.477510  1.400000 -4.873439   
    1955  -0.225249     2.031935    0.476961  1.505632  1.400000 -2.927289   
    1956   0.050236     2.059166    0.477863  1.534980  1.400000 -1.997915   
    1957  -0.037443     2.078468    0.532640  1.548565  1.620000 -2.384903   
...             ...          ...         ...       ...       ...       ...   
SWE 2015  -0.017783     3.941785    2.733937  2.552606 -0.289167  0.149843   
    2016  -0.023360     3.953127    2.735742  2.548431 -0.655000  1.465335   
    2017   0.007405     3.969385    2.743045  2.555912 -0.695000  1.650562   
    2018   0.056920     3.982214    2.762205  2.557299 -0.685800  1.359689   
    2019   0.053172     3.999731    2.764608  2.572266 -0.420800  1.337586   

          debt_gdp  
iso year            
BEL 1953   68.6405  
    1954   69.6069  
    1955   68.2519  
    1956   65.2228  
    1957   62.6271  
...            ...  
SWE 2015   43.7387  
    2016   42.2554  
    2017   40.7310  
    2018   38.9087  
    2019   34.8980  

[797 rows x 7 columns]

In [22]:
df = df.reset_index()

In [23]:
df.to_csv(Path.cwd().parent / "data processed" / "nato_dataset.csv", index=False)

In [24]:
years_per_iso = (
    df[["iso", "year"]]
    .drop_duplicates()
    .groupby("iso")["year"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name="n_years")
)

years_per_iso

,iso,n_years
0,FRA,70
1,GBR,70
2,NOR,70
3,PRT,70
4,ITA,69
5,BEL,67
6,DNK,66
7,ESP,66
8,NLD,64
9,CHE,63


In [31]:
df.sort_values("mil_delta").tail(20)

,iso,year,mil_delta,log_rgdp_pc,log_gov_pc,log_tfp,tbill,def_gdp,debt_gdp
648,NOR,2001,0.373087,3.808376,2.712792,2.792089,5.810000,13.881264,27.422810
537,NLD,1960,0.383382,2.144825,0.737172,1.879977,3.738523,0.323113,66.740300
425,GBR,1981,0.397563,-4.179219,-5.295037,2.163813,12.980001,4.195020,44.778000
773,SWE,1996,0.422249,3.450817,2.620034,2.251431,2.363100,-1.812181,73.064893
532,ITA,2019,0.423733,-3.645908,-4.872402,2.384717,-0.060000,-1.739308,134.563000
137,DNK,1961,0.431461,-4.322348,-5.997611,1.874174,6.625000,-0.931691,16.430769
209,ESP,1967,0.445716,1.857793,-0.223196,1.807122,3.000000,-0.022457,16.963472
396,GBR,1952,0.586506,-5.045131,-6.281548,1.618439,2.200000,4.212536,180.858201
683,PRT,1966,0.610894,1.368157,-0.681594,1.436674,2.750000,-0.626329,22.382814
265,FIN,1961,0.676997,2.304470,0.891576,1.359615,4.160000,-6.480776,10.130000
